In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Feb 12 16:33:24 2023

Exploration d'un labyrinthe

Université Lumière Lyon 2
L2 Informatique
2022-2023

@author: jvelcin
"""

#import os
import numpy as np
import random

# 1. initialisation du labyrinthe

#filename = "monlaby_1.txt"
filename = "monlaby_3.txt"

with open(filename) as f:  
    monlaby_read = [line for line in f.readlines()]

monlaby = {}
monlaby["numcol"] = np.min([len(s) for s in monlaby_read])
monlaby["numlig"] = len(monlaby_read)
monlaby["map"] = np.zeros((monlaby["numlig"], monlaby["numcol"]))
for i in range(monlaby["numlig"]):
    for j in range(monlaby["numcol"]):
        monlaby["map"][i, j] = monlaby_read[i][j]

# 2. initialisation du héros

aventurier = {}
aventurier["position"] = [6, 0]
aventurier["symbol"] = "A"
aventurier["etat"] = 0 # pas de joyau

# le joyau
joyau = {}
#joyau["position"] = [7, 4]
joyau["position"] = [1, 8] # pour la map 3
joyau["symbol"] = "*"

# pour le moment, un seul personnage = l'aventurier
personnages = [aventurier]

#affichage texte de la carte, des personnages et du joyau
def print_laby(m, p, j):
    s = ""
    for i in range(m["numlig"]):
        for j in range(m["numcol"]):
            case = "."
            if m["map"][i,j] == 1:
                case = "X"
            else:
                if joyau["position"] == [i,j]:
                    case = joyau["symbol"]
                
                for perso in p:
                    if perso["position"] == [i,j]:
                        case = perso["symbol"]
            s += case + "\t"
        s += "\n"
    return s

# les quatre mouvements possibles à priori
# (nord, sud, ouest, est)
moves = [
    (-1, 0),
    (+1, 0),
    (0, -1),    
    (0, +1)
    ]

# retourne toutes les nouvelles positions atteignables à priori (sans vérification)
def get_possible_positions(p, m):
    res = []
    r = [0,0]
    for move in m:
        r[0] = p[0] + move[0]
        r[1] = p[1] + move[1]
        res.append(r.copy())
    return res

# est-ce que la position p est autorisée sur la map m ?
def is_position_allowed(p, m):
    if p[0]<0:
        return False
    if p[0]>(m["numlig"]-1):
        return False
    if p[1]<0:
        return False
    if p[1]>(m["numcol"]-1):
        return False
    if m["map"][p[0],p[1]] != 0:
        return False
    return True

# fonctions d'exploration

# ajoute la nouvelle position au début de la liste
def ajout_FIFO(to_explore, p):
    if p not in to_explore:
        to_explore = [p] + to_explore
    return to_explore 

# ajoute la nouvelle position à la fin de la liste
def ajout_LIFO(to_explore, p):
    if p not in to_explore:
        to_explore = to_explore + [p]
    return to_explore     

# sélectionne la prochaine position à explorer
#  (l'option de choix aléatoire rajoute un peu de complication car il faut
#  concaténer les position avant et après celle qui a été choisie...)
def prochaine_pos(to_explore, alea=False):
    #print("avant " + str(to_explore))
    if alea:  # tirage aléatoire 
        d = random.randint(0, len(to_explore)-1)
    else:
        d = 0
    p = to_explore[d]
    if d>0:
        gauche = to_explore[0:d]
    else:
        gauche = []
    #print("gauche " + str(gauche))
    if d<(len(to_explore)-1):
        droite = to_explore[d+1:len(to_explore)]
    else:
        droite = []
    #print("droite " + str(droite))
    to_explore_next = gauche + droite
    #print("reste " + str(to_explore_next))
    return p, to_explore_next

# boucle principale

choix = ""
to_explore = [] # liste des positions qui restent à explorer
memory = [aventurier["position"]] # pour vérifier qu'on ne boucle pas à l'infini s'il y a des cycles

cout = 0 # cout de la recherche

while (choix != "Q"):
    print(f"aventurier en : {aventurier['position']}")
    print(print_laby(monlaby, personnages, joyau))
    choix = input("(Q to quit) > ")
    if choix=="Q":
        break
    pos_possible = []
    for p in get_possible_positions(aventurier["position"], moves):
        if is_position_allowed(p, monlaby):
            pos_possible.append(p)
            if p not in memory:
                #to_explore = ajout_LIFO(to_explore, p)
                to_explore = ajout_FIFO(to_explore, p)
    if len(pos_possible) == 0:
        print("L'aventurier est coincé !")
        break
    if len(to_explore) == 0:
        print("L'aventurier a déjà tout exploré !")
        break
    else:
        p, to_explore = prochaine_pos(to_explore, alea=True)
        aventurier["position"] = p
        memory.append(p)
        cout += 1
        if (aventurier["position"] == joyau["position"]):
            print(f"L'aventurier a trouvé le joyau en {cout} étapes!")
            aventurier["etat"] = 1 # prise du joyau
            break
    



aventurier en : [6, 0]
X	X	X	X	X	X	X	X	X	X	
X	X	X	.	X	.	.	X	*	X	
X	X	.	.	.	.	X	.	.	X	
X	X	.	X	X	X	X	.	X	X	
X	X	.	.	.	.	.	.	X	X	
X	X	.	X	X	.	X	.	X	X	
A	.	.	.	X	.	X	.	X	X	
X	X	X	.	X	X	X	.	.	X	
X	X	X	.	.	.	X	X	.	X	
X	X	X	X	X	X	X	X	X	X	

(Q to quit) > 
aventurier en : [6, 1]
X	X	X	X	X	X	X	X	X	X	
X	X	X	.	X	.	.	X	*	X	
X	X	.	.	.	.	X	.	.	X	
X	X	.	X	X	X	X	.	X	X	
X	X	.	.	.	.	.	.	X	X	
X	X	.	X	X	.	X	.	X	X	
.	A	.	.	X	.	X	.	X	X	
X	X	X	.	X	X	X	.	.	X	
X	X	X	.	.	.	X	X	.	X	
X	X	X	X	X	X	X	X	X	X	

(Q to quit) > 
aventurier en : [6, 2]
X	X	X	X	X	X	X	X	X	X	
X	X	X	.	X	.	.	X	*	X	
X	X	.	.	.	.	X	.	.	X	
X	X	.	X	X	X	X	.	X	X	
X	X	.	.	.	.	.	.	X	X	
X	X	.	X	X	.	X	.	X	X	
.	.	A	.	X	.	X	.	X	X	
X	X	X	.	X	X	X	.	.	X	
X	X	X	.	.	.	X	X	.	X	
X	X	X	X	X	X	X	X	X	X	

(Q to quit) > 
aventurier en : [6, 3]
X	X	X	X	X	X	X	X	X	X	
X	X	X	.	X	.	.	X	*	X	
X	X	.	.	.	.	X	.	.	X	
X	X	.	X	X	X	X	.	X	X	
X	X	.	.	.	.	.	.	X	X	
X	X	.	X	X	.	X	.	X	X	
.	.	.	A	X	.	X	.	X	X	
X	X	X	.	X	X	X	.	.	X	
X	X	X	.	.	.	X	X	.	X	
X	X	X	X	X	X	X	X	X	X	

(Q to quit) > 
aven